In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install lumnisai

In [3]:
from IPython.display import Markdown
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import os

import lumnisai


load_dotenv()


def display_markdown(text):
    display(Markdown(text))

# Create client
lumnisai_agent = lumnisai.AsyncClient()

# List users
users = await lumnisai_agent.list_users(page_size=50)

# Check if alice@test-acme.one exists in users
alice_user = next((user for user in users.users if user.email == "alice@test-acme.one"), None)

# Create user if not found
if alice_user is None:
    user = await lumnisai_agent.create_user( email="alice@test-acme.one", first_name="Alice", last_name="Doe" )

# Print users
print("Existing Users:")
for user in users.users:
    print("    ", user.email, user.id)

print("="*50)

# List API keys
api_keys = await lumnisai_agent.list_api_keys()
print("Existing API keys:")
for api_key in api_keys:
    print("    ", api_key.provider, "Exists - Created at", api_key.created_at)

print("="*50)

# Add API keys
await lumnisai_agent.add_api_key(lumnisai.ApiProvider.OPENAI_API_KEY, os.getenv("OPENAI_API_KEY"))
await lumnisai_agent.add_api_key(lumnisai.ApiProvider.EXA_API_KEY, os.getenv("EXA_API_KEY"))

preferences = await lumnisai_agent.get_model_preferences()
print(preferences)

user_email = "moribajaye@gmail.com" 

# List connections
print("="*50)
connections = await lumnisai_agent.list_connections(user_email)
print(connections)

# # List tools
# print("="*50)
# tools = await lumnisai_agent.get_integration_tools(user_email)
# print(tools)

print("Setup Complete!")

Existing Users:
     lumnistrading@gmail.com 8f99e82d-ca41-4e51-84c8-54eee4588d68
     peytonjohnson710@gmail.com f065a840-2005-4181-93d7-40544666cafd
     kabirjaiswal30@gmail.com 559a7a0c-e1fc-4a44-80e3-e9e1ef8fff2f
     aakash@secondaxis.ai 5f55ac40-841e-4c53-832e-d21b2e6f313e
     ajaye12@illinois.edu a312cf13-0693-4713-9433-4c139b0da185
     moribajaye@gmail.com 8fc1a34c-b335-4f03-a25c-8dadba04dab1
     ajaye2@illinois.edu 4bd0c7f3-9f13-481d-a673-6a03c8945c6c
     abujaye@gmail.com b33224a4-8700-4427-be85-7baedacd0115
     mosompo08@gmail.com ab778a36-4cc0-4aa3-9ac7-087307395e5a
     mikeszczerba0@gmail.com a9625b3c-c4a1-4b82-addf-2b0c7c3942be
     alice@test-acme.one 8d5627e1-4f66-4676-99bf-3d5bedcd45f4
Existing API keys:
     E2B_API_KEY Exists - Created at 2025-09-04T02:48:00.873043+00:00
     EXA_API_KEY Exists - Created at 2025-08-17T19:19:54.791579+00:00
     OPENAI_API_KEY Exists - Created at 2025-08-17T19:19:50.588078+00:00
Model Preferences (Tenant: 7ed13659-53d8-428d-b86

In [4]:
mcp_serrver_name = "Supabase-mcp-server"

mcp_servers = await lumnisai_agent.list_mcp_servers()
supabase_server_exists = next((server for server in mcp_servers.servers if server.name == "Supabase-mcp-server"), None)

if supabase_server_exists is None:
    supabase_server = await lumnisai_agent.create_mcp_server(
        name=f"Supabase-mcp-server",
        transport="stdio",
        scope="user",
        user_identifier=user_email,
        description="Supabase",
        command="npx",
        args=["-y", "@supabase/mcp-server-supabase@latest", "--project-ref=rcxermhxsgrxyfvpozcb"],
        env={
            "SUPABASE_ACCESS_TOKEN": os.environ.get("SUPABASE_ACCESS_TOKEN"),
        },
    )
    print(f"Supabase MCP server created: {supabase_server.name}")
else:
    print(f"Supabase MCP server already exists: {supabase_server_exists.name}")



Supabase MCP server already exists: Supabase-mcp-server


In [5]:

# Streaming response  
async for update in await lumnisai_agent.invoke(""" 
                                                What are the latest trends in AI agents and machine learning? 
                                                What are agentic models? 
                                                What are the latest papers on agentic models and agentic AI?. 
                                                Send email to moriba.jaye@gmail.com.
                                                """, stream=True, user_id=user_email):
    print(f"{update.state.upper()} - {update.message}")

display(Markdown(update.output_text))

Response ID: 9c50967a-a889-49c9-a4da-123abda91319
PROCESSING - Agent thinking...
PROCESSING - Analyzing request and planning approach...
PROCESSING - Researching and synthesizing the latest trends in AI agents, explaining agentic models, and summarizing recent academic papers on agentic AI.
PROCESSING - Sending an email with the synthesized AI agent trends summary to moriba.jaye@gmail.com.
COMPLETED - Response completed successfully.
COMPLETED - Task completed successfully


Your requested research summary on the latest trends in AI agents, explanations of agentic models, and recent academic papers has been compiled and successfully emailed to moriba.jaye@gmail.com with the subject line:

“2024-2025 Snapshot: What You Need to Know About Agentic AI & Agentic Models”

Please check your inbox (and spam folder if necessary) to review the full HTML-formatted content, complete with references and key take-aways.

In [6]:
task = """
List tables in the --project-ref=rcxermhxsgrxyfvpozcb project?

There is a table called "temp" in the public schema. Please return all columns and rows from the public.temp table.

Step 1.: Start by listing all available schemas
Step 2.: Then list the columns and rows from the temp table.
Step 3.: Return the columns and rows from the temp table.
"""

async for update in await lumnisai_agent.invoke(task, stream=True, user_id=user_email):
    print(f"{update.state.upper()} - {update.message}")

display(Markdown(update.output_text))


Response ID: da58bd55-564b-4b58-b0cb-14e68921b9d3
PROCESSING - Agent thinking...
PROCESSING - Analyzing request and planning approach...
PROCESSING - Connecting to project rcxermhxsgrxyfvpozcb and listing all database schemas.
PROCESSING - Retrieve all columns and rows from the public.temp table in rcxermhxsgrxyfvpozcb.
COMPLETED - Response completed successfully.
COMPLETED - Task completed successfully


Schemas  
auth, extensions, graphql, graphql_public, information_schema, pg_catalog, pg_toast, pgbouncer, public, realtime, storage, vault  

public.temp columns  
- id (bigint)  
- created_at (timestamp with time zone)  
- abu (text)  

public.temp rows  
| id    | created_at (UTC)                 | abu  |
|-------|----------------------------------|------|
| 232 322 | 2025-09-07 03:24:33.044433 +00 | 2432 |
| 23      | 2025-09-07 03:24:38.723035 +00 | dfdf |

In [7]:

task = """
Read the top 1000 rows of the 9,000+ Investors spreadsheet and identify which ones would be good for Lumnis AI? We're building an API platform for AI agents and looking for pre-seed/incubator stage investors.

The name of the spreadsheet is "9,000+ Investors".
"""

async for update in await lumnisai_agent.invoke(task, stream=True, user_id=user_email):
    print(f"{update.state.upper()} - {update.message}")

display(Markdown(update.output_text))

Response ID: b3484b0f-b56f-4b68-bc11-827576416804
PROCESSING - Agent thinking...
PROCESSING - Analyzing request and planning approach...
PROCESSING - Locate the '9,000+ Investors' spreadsheet and pull document metadata, sheet details, and sample rows.
PROCESSING - Analyze investors in rows 1–500 for suitability to Lumnis AI and produce partial ranked shortlist.
PROCESSING - Analyze investors in rows 501–1000 and produce second partial shortlist for Lumnis AI.
PROCESSING - Merge the two partial investor lists and generate final ranked shortlist of best-fit investors with metadata.
PROCESSING - Aggregate the two partial lists above into a final ranked shortlist of 10–20 investors with metadata.
COMPLETED - Response completed successfully.
COMPLETED - Task completed successfully


**Best-Fit Investors for Lumnis AI (API platform for AI agents)**  
Pre-seed / seed accelerators, angels & micro-VCs ranked by sector fit, stage, U.S. location and recent activity.

| # | Investor | Contact | Stage / Type | Core Sectors | Typical Check | HQ | Score¹ |
|---|----------|---------|--------------|--------------|---------------|----|--------|
| 1 | Y Combinator | trevor@ycombinator.com | Accelerator – Pre-seed | DevTools, API, AI infra | ≈ $120 k | Mountain View CA | 94 |
| 2 | Zetta Venture Partners | mark@zettavp.com | Seed VC | Enterprise AI / infra | $1–5 M | San Francisco CA | 91 |
| 3 | Techstars | david.cohen@techstars.com | Accelerator – Pre-seed | Platforms & APIs | $100–200 k | Boulder CO | 90 |
| 4 | Work-Bench | jon@work-bench.com | Seed VC | Big-data, DevOps | $0.5–1 M | New York NY | 88 |
| 5 | Alchemist Accelerator | danielle@alchemistaccelerator.com | Accelerator (B2B) | DevOps, Cloud infra | $60 k | San Francisco CA | 88 |
| 6 | Auren Hoffman (angel) | auren@safegraph.com | Angel | Data / API infra | $10–250 k | San Francisco CA | 87 |
| 7 | Ilya Sukhar (angel) | ilya@matrixpartners.com | Angel | Dev platforms, APIs | $25–250 k | San Francisco CA | 86 |
| 8 | Initialized Capital | garry@initialized.co | Micro-VC Seed | DevTools, Infra | $0.5–2 M | San Francisco CA | 85 |
| 9 | DCVC (Data Collective) | matt@dcvc.com | Micro-VC Seed–A | AI / data infra | $1–5 M | Palo Alto CA | 85 |
|10 | Wildcat Ventures | kbarr@mdv.com | Seed VC | SaaS platforms | $0.25–15 M | San Mateo CA | 82 |
|11 | TechOperators | said@techoperators.com | Seed VC | Cloud / infra | $2–4 M | Atlanta GA | 81 |
|12 | Boost VC | adam@boost.vc | Accelerator – Pre-seed | Dev ecosystems | $10–50 k | San Mateo CA | 80 |
|13 | Bowery Capital | mike.brown@bowerycap.com | Seed VC | B2B SaaS | $0.25–3 M | New York NY | 80 |
|14 | Ulu Ventures | miriam@uluventures.com | Seed VC | Software, data | $0.5–1 M | Palo Alto CA | 79 |
|15 | Right Side Capital | dave@rightsidecapital.com | Micro-VC Pre-seed | DevTools | $30–200 k | San Francisco CA | 78 |
|16 | Venture51 | ryan@venture51.com | Early-stage VC | Software infra | $0.5–1 M | Scottsdale AZ | 78 |
|17 | Founder Collective | david@foundercollective.com | Seed VC | B2B SaaS | — | Cambridge MA | 77 |
|18 | F50 | david@f50.io | VC + syndicate | AI / software | $5 M | San Francisco CA | 77 |
|19 | Caffeinated Capital | rt@caffeinatedcapital.com | Seed VC | Data / SaaS | — | San Francisco CA | 77 |
|20 | True Ventures | jim@trueventures.com | Seed VC | SW & HW infra | $0.25–20 M | Palo Alto CA | 75 |

¹Composite rubric: 40 % sector alignment • 30 % stage fit • 20 % U.S. geography • 10 % recent activity/notes.

Key take-aways  
• All 20 investors routinely back developer-focused, infrastructure or AI-platform startups at pre-seed/seed.  
• Mix of top accelerators, high-throughput micro-VCs, specialized AI funds and renowned angels gives multiple entry points.  
• Geographic spread favors Bay Area & NYC but includes Boulder, Atlanta and Scottsdale for syndication diversity.

Spreadsheet details  
• Sheet: “9,000+ Investors” (ID 1730MLTD7qXP4E6IfbnhwICugxDSbkO1KJpLiAUKdQvk)  
• Rows processed: 1–1 000 (data rows 3-1 002) out of 9  278 total  
• Column headers used: Firm, Website, Description, Contact, Phone Number, Title, Email, Total Investments, Active Investments, Dry Powder, Notes, Notes 2, Preferred Sectors, Preferred Investment Size, Preferred Transaction Sizes, Preferences, Date of Last Fund Closed, AUM, Type, HQ  

Top exclusion reasons for near-miss investors  
1. Score < 75 after rubric weighting.  
2. Late-stage focus or unclear activity at pre-seed/seed.  
3. Non-U.S. HQ or consumer-only sector alignment.

This shortlist gives Lumnis AI a focused set of high-probability investors to approach for its developer-centric AI-agent API platform.

In [8]:
query =  """
Here is my investment thesis for SBUX:

China expansion: Opening more than 500 new stores
Profitability: Achieving operating margin above 15%
Cost stability: Coffee input costs remaining stable year-over-year
Growth target: Revenue growth exceeding 20% year-over-year in 2025

ANALYSIS REQUEST:
- Primary focus: Validate or invalidate each point of my thesis above
- Provide a clear assessment of how many thesis points are supported vs. contradicted by recent evidence
- Score each thesis point as: VALIDATED, DEGRADED, or NEUTRAL based on recent information

Please analyze recent news, market information, and analyst reports for SBUX from the past week and provide:
1. Detailed analysis of how recent developments impact each thesis point
2. Clear scoring/validation status for each of the 4 thesis components
3. Overall thesis strength assessment based on recent evidence

In addition to the thesis validation, include a break over view of all the analysis and information about the SBUX stock. 
Include this in the beginning of the report.
Then start looking at the thesis points. 

For all information used to validate/invalidate the thesis it must have been published in the past week.
If the information is not published in the past week then do not use it to validate/invalidate the thesis. This is redundant information we already have.
For all data points used, specify the exact date in which it was released.


Structure the report with the following sections in order:
1. Overview: SBUX State-of-the-Stock 2025-07-12 - Should contain all the information and analysis based on the latest news, market information, and analyst reports, released in the past week. (Note. Only include information that is published in the past week.)
1. 'Thesis Component-by-Component Assessment' - A comprehensive markdown table with validation status, latest evidence, and key risks for each thesis point
2. 'Detailed Analysis of Each Thesis Point' - In-depth analysis of each component with supporting facts and scoring rationale
3. 'Thesis Validation Summary Table' - Concise table summarizing validation status and outlook for each thesis point
4. 'Research Debate Key Conclusions' - Summary of key findings from the research team's debate rounds
5. 'Trading Analysis & Technical Assessment' - Technical analysis and timing considerations from the trader
6. 'Risk Scenario Analysis' - Comprehensive risk assessment and scenario modeling
7. 'Opportunity Assessment & Executive Summary' - Final assessment with actionable insights and a 'Comprehensive Investment Analysis Summary Table' organizing all key findings

Make sure to cite all the sources of information in the report following proper citation standards. 
Include the publication name, article title, date, and URL when available. Use inline citations throughout the text. 

Make sure to include all the information in the report. Do not leave any information out.

For all tools you use filter with the following dates:
TODAY is 2025-07-12
START_DATE is 2025-07-05 
"""

async for update in await lumnisai_agent.invoke(query, stream=True, user_id=user_email):
    print(f"{update.state.upper()} - {update.message}")

display(Markdown(update.output_text))

Response ID: 1e742231-a405-45a3-bcc1-484b3a1e7d3d
PROCESSING - Agent thinking...
PROCESSING - Analyzing request and planning approach...
PROCESSING - Gather recent (2025-07-05 to 2025-07-12) news, reports, and technical commentary about Starbucks (SBUX) focusing on China expansion, profitability, cost stability, revenue growth, and stock performance. Provide 10-20 relevant items with citations and summaries.


CancelledError: 

In [ ]:
"""
Example demonstrating how to use structured output with the Lumnis SDK.

The SDK supports two ways to specify response format:
1. Pass a Pydantic model class directly (recommended)
2. Pass a JSON Schema dictionary
"""
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: Optional[str] = None

class BusinessInfo(BaseModel):
    name: str = Field(description="Business name")
    category: str = Field(description="Type of business")
    address: Address
    phone: Optional[str] = None
    website: Optional[str] = None
    rating: Optional[float] = Field(None, ge=0, le=5)


# Example: Complex structure with nested models
response_structured = await lumnisai_agent.invoke(
    messages=[{"role": "user", "content": "Tell me about the Louvre Museum. Get the address, and as much business information as possible (e.g. category, phone, website, rating)."}],
    response_format=BusinessInfo,  # Pass the model class directly
    response_format_instructions="Include all available details",
    user_id=user_email
)

if response_structured.structured_response:
    museum = BusinessInfo(**response_structured.structured_response)
    print(f"Museum: {museum.name}")
    print(f"Location: {museum.address.city}, {museum.address.country}")

    print(json.dumps(response_structured.structured_response, indent=4))



NameError: name 'BaseModel' is not defined

In [ ]:
response = await user_scoped_client.invoke(
    prompt="What are the latest trends in AI agents and machine learning? ",
    show_progress=True
    )
display_markdown(response.output_text)

Response ID: 9f8f8bb6-6588-4e11-b874-0c9da6e8773e
PROCESSING: Starting agent...
PROCESSING: Research the most recent (2024-2025) trends in AI agents, referencing reputable sources like academic papers, blogs, and news.
PROCESSING: Research the most recent (2024-2025) trends in machine learning, providing relevant references from academic and reputable industry sources.
PROCESSING: Analyze and synthesize the latest trends in AI agents and machine learning (2024-2025), highlighting overlaps, unique developments, and emerging directions, with specific references.
PROCESSING: Write a comprehensive report (with executive summary, detailed trends, synthesis, and references) on the latest (2024-2025) trends in AI agents and machine learning.
PROCESSING: Send a notification message to stakeholders to inform them the AI agents and machine learning trends report is finished, including the report content or a link to it.
COMPLETED: Response completed successfully.


# Report: Latest Trends in AI Agents and Machine Learning (2024–2025)

---

## Executive Summary

AI agents and machine learning (ML) are converging to power autonomous, adaptable, and collaborative digital intelligence. In 2025, AI agents are widely deployed across industries, leveraging breakthroughs in multimodal, generative models. ML fuels this ecosystem with scalable, explainable, and privacy-preserving approaches. Key trends include explosive adoption, advances in autonomy and multimodality, democratization via no-code/low-code tools, human-centered design, and the active emergence of standards for reliability and governance.

---

## 1. AI Agents: Key Trends
- **Mainstream Deployment:** Over 99% of enterprise developers use or explore AI agents across core business functions (finance, healthcare, supply chain, customer service). Market projected to reach $7.92B in 2025 and $236B by 2034.  
  *[IBM, Precedence Research, Master of Code]*
- **Complex, Autonomous, Multi-Agent Systems:** Agents now coordinate with each other using new agent-to-agent protocols (A2A, MCP) and orchestrate multifaceted workflows.  
  *[Capgemini, MIT TechReview]*
- **Next-Gen Platforms:** Major tech players have released agent SDKs and marketplaces (Microsoft Agent Store, Google Gemini Console, AWS, Salesforce, Nvidia).  
  *[CRN, Daily AI Agent News]*
- **Multimodal & Embodied Agents:** Agents process and act on text, vision, speech, and physical environments (e.g., GPT-4o, Gemini, Nvidia Eureka).  
  *[MIT TechReview, Medium]*
- **Human-Augmentation:** Copilot-style agents thrive, especially where safety, compliance, or regulation is key.  
  *[IBM]*
- **Challenges:** Reliability, security, privacy, and energy demands are leading concerns.  
  *[LangChain Survey, MIT TechReview, Deloitte]*

---

## 2. Machine Learning: Key Trends
- **Generative & Multimodal Models:** Foundation models (GPT-4o, Gemini, LLaMA) integrate text, image, video, and audio, boosting agentic and general ML use-cases.  
  *[Machine Learning Mastery, LinkedIn]*
- **Autonomous/Agentic AI:** ML drives ever more capable software agents via advances in reinforcement learning, large context, and self-supervision.  
  *[Machine Learning Mastery, Medium]*
- **Explainable, Ethical, and Responsible AI:** Growing demand for fairness, transparency, and compliance in ML, supported by regulatory moves in healthcare, finance, and law.  
  *[GeeksforGeeks, Medium]*
- **Federated & Edge ML:** Privacy-preserving, decentralized deployment of ML models, vital for sensitive industries like healthcare, IoT, and finance.  
  *[Medium, GeeksforGeeks]*
- **Data-Centric AI:** Synthetic datasets, AutoML tools, and low-code/no-code platforms lower barriers for non-experts.  
  *[Medium, LinkedIn]*
- **Quantum and Causal ML:** Early-stage hybrid quantum-classical and causal inference techniques are emerging.  
  *[LinkedIn, Medium]*
- **Real-Time/Continual Learning:** Models adapt in real-time from streaming data, powering dynamic business and industrial apps.  
  *[Medium, LinkedIn]*

---

## 3. Synthesis & Comparative Analysis

### Overlapping Trends
- **Autonomy & Collaboration:** Both fields are rapidly converging on self-directed, context-adaptive, multi-agent architectures.
- **Multimodality & Generativity:** Foundation models empower agents with broad, generative abilities across data types.
- **Democratization:** Low-code/no-code platforms and marketplaces drive use by domain experts and non-coders.
- **Human-centered, Responsible Intelligence:** Emphasis on human oversight, safety, reliability, and auditability.
- **Common Challenges:** Both struggle with output honesty (hallucinations), compliance, privacy, and energy impact.

### Unique Developments
- **AI Agents:** Focus on workflow automation, agent-to-agent protocols, and agent application marketplaces.
- **Machine Learning:** Advances in foundation models (>10T parameters), continual/self-supervised learning, and novel hardware.

### Emerging Directions
- Continual-learning and end-to-end differentiable agent architectures.
- Standardization enabling ML models and agents to interoperate seamlessly.
- "Ethical agent" frameworks for transparency, fairness, and accountability.

---

## Reference List

1. [IBM: AI Agents in 2025—Expectations vs. Reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality)
2. [Precedence Research: AI Agents Market Size Worth USD 236B by 2034](https://www.globenewswire.com/news-release/2025/07/02/3109307/0/en/AI-Agents-Market-Size-Worth-USD-236-03-Billion-by-2034-Fueled-by-Machine-Learning-and-Natural-Language-Processing-Advances.html)
3. [Master of Code: 150+ AI Agent Statistics (July 2025)](https://masterofcode.com/blog/ai-agent-statistics)
4. [Capgemini/CNBC: AI Agent Collaboration in 2025](https://www.cnbc.com/2024/07/22/ai-that-can-talk-with-other-ai-will-launch-in-2025-capgemini-predicts.html)
5. [MIT Technology Review: What are AI agents? (July 2024)](https://www.technologyreview.com/2024/07/05/1094711/what-are-ai-agents/)
6. [CRN: 10 Hottest Agentic AI Tools Of 2025](https://www.crn.com/news/ai/2025/10-hottest-agentic-ai-tools-and-agents-of-2025-so-far)
7. [AI Agent Store: Daily AI Agent News, May 2025](https://aiagentstore.ai/ai-agent-news/2025-may)
8. [Medium (Edwin Lisowski): AI Agents Take the Lead: 2025](https://medium.com/@elisowski/a-list-of-ai-agents-set-to-dominate-in-2025-028f975c5b99)
9. [Deloitte: TMT 2025 Predictions](https://www2.deloitte.com/us/en/insights/industry/technology/technology-media-and-telecom-predictions/2025/autonomous-generative-ai-agents-still-under-development.html)
10. [Machine Learning Mastery - 7 Machine Learning Trends to Watch in 2025](https://machinelearningmastery.com/7-machine-learning-trends-2025/)
...and more listed in the detailed findings section.

---

**For further detail, see referenced academic articles and industry reports. The convergence of AI agents and ML is defining the new era of adaptive, intelligent digital systems across the global economy.**